In [38]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [39]:
import warnings

warnings.filterwarnings("ignore")

In [40]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

In [41]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=2)

In [42]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
print(
    "Cross-validation scores:\n{}".format(
        cross_val_score(logreg, iris.data, iris.target, cv=kfold)
    )
)

Cross-validation scores:
[0.29333333 0.33333333]


In [150]:
import pandas as pd
from sklearn.model_selection import KFold, RepeatedKFold

data = pd.DataFrame(
    [
        ["red", "strawberry"],  # color, fruit
        ["red", "strawberry"],
        ["red", "strawberry"],
        ["red", "strawberry"],
        ["red", "strawberry"],
        ["yellow", "banana"],
        ["yellow", "banana"],
        ["yellow", "banana"],
        ["yellow", "banana"],
        ["yellow", "banana"],
    ]
)

X = data[0]

# KFold
for train_index, test_index in KFold(n_splits=3, shuffle=True, random_state=1).split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

print()
# RepeatedKFold
for train_index, test_index in RepeatedKFold(
    n_splits=3, n_repeats=3, random_state=1
).split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [0 1 3 5 7 8] TEST: [2 4 6 9]
TRAIN: [2 4 5 6 7 8 9] TEST: [0 1 3]
TRAIN: [0 1 2 3 4 6 9] TEST: [5 7 8]

TRAIN: [0 1 3 5 7 8] TEST: [2 4 6 9]
TRAIN: [2 4 5 6 7 8 9] TEST: [0 1 3]
TRAIN: [0 1 2 3 4 6 9] TEST: [5 7 8]
TRAIN: [1 2 4 6 7 8] TEST: [0 3 5 9]
TRAIN: [0 1 3 5 6 7 9] TEST: [2 4 8]
TRAIN: [0 2 3 4 5 8 9] TEST: [1 6 7]
TRAIN: [0 1 2 4 6 7] TEST: [3 5 8 9]
TRAIN: [2 3 4 5 7 8 9] TEST: [0 1 6]
TRAIN: [0 1 3 5 6 8 9] TEST: [2 4 7]
